In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/HDDS/Data

In [ ]:
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
import time as t
from PIL import Image, ImageEnhance

!sudo apt-get install imagemagick imagemagick-doc
!sudo apt-get install imagemagick
!sudo apt-get install unrar

Reading package lists... Done
Building dependency tree       
Reading state information... Done
imagemagick is already the newest version (8:6.9.7.4+dfsg-16ubuntu6.11).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unrar is already the newest version (1:5.5.8-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
#for k in range(2):
image = cv2.imread("1.png")
plt.imshow(image)
plt.show()

#Make Image Grayscale
print(image.shape)
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imwrite('MainImage.png',gray_image)
#m = (0,0,0) 
print(image.shape)
MainImage = cv2.imread("MainImage.png")


In [ ]:
#fix Brightness
from PIL import Image, ImageEnhance

for i in range(13):

  img = Image.open("/content/drive/MyDrive/HDDS/Data/English/jpgs/"+str(i+1)+".jpg")
  #display(img)
  enhancer = ImageEnhance.Contrast(img)

  factor = 3
  im_s_1 = enhancer.enhance(factor)
  im_s_1.save("/content/drive/MyDrive/HDDS/Data/English/jpgEnhanced/"+str(i+1)+".png");

In [ ]:
 #Just converting
 img = Image.open("/content/drive/MyDrive/HDDS/Data/English/jpgs/ocr-4.jpeg")
 img.save("/content/drive/MyDrive/HDDS/Data/English/jpgEnhanced/11.png")

In [ ]:
#filters:

"""
1- Gaussian Noise
2- Random Noise
3- Brightness and Sharpness
"""

"""
Parameters
----------
image : ndarray
    Input image data. Will be converted to float.
mode : str
    One of the following strings, selecting the type of noise to add:

    'gauss'     Gaussian-distributed additive noise.
    'poisson'   Poisson-distributed noise generated from the data.
    's&p'       Replaces random pixels with 0 or 1.
    'speckle'   Multiplicative noise using out = image + n*image,where
                n is uniform noise with specified mean & variance.
"""

import numpy as np
import os
import cv2

varr=[0.1,0.3] #Can add more values like: 0.25,0.15
SPIn=[0.5,0.4] #Can add more values like: 0.55,0.6,0.8

def noisy(SPIn,varIn,noise_typ,image):
  if noise_typ == "gauss":
    row,col,ch= image.shape
    mean = 0
    var = varIn
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss
    return noisy
  elif noise_typ == "s&p":
    row,col,ch = image.shape
    s_vs_p = SPIn
    amount = 0.004
    out = np.copy(image)
    # Salt mode
    num_salt = np.ceil(amount * image.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt))
            for i in image.shape]
    out[coords] = 1

    # Pepper mode
    num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper))
            for i in image.shape]
    out[coords] = 0
    return out
  elif noise_typ == "poisson":
    vals = len(np.unique(image))
    vals = 2 ** np.ceil(np.log2(vals))
    noisy = np.random.poisson(image * vals) / float(vals)
    return noisy
  elif noise_typ =="speckle":
    row,col,ch = image.shape
    gauss = np.random.randn(row,col,ch)
    gauss = gauss.reshape(row,col,ch)        
    noisy = image + image * gauss
    return noisy

In [ ]:
#Testing
image = cv2.imread("test.jpg")
plt.imshow(image)
plt.show()


test=noisy("gauss",image)
plt.imshow(test)
plt.show()
cv2.imwrite('test1.png',test)

test=noisy("s&p",image)
plt.imshow(test)
plt.show()
cv2.imwrite('test2.png',test)

test=noisy("poisson",image)
plt.imshow(test)
plt.show()
cv2.imwrite('test3.png',test)

test=noisy("speckle",image)
plt.imshow(test)
plt.show()
cv2.imwrite('test4.png',test)

In [ ]:
import os
pwd ="/content/drive/MyDrive/HDDS/Data/English/Augmented/"
l=(3507, 2550,3)
cnt=0
for i in range(1,188):
  image = cv2.imread(pwd+str(i)+'.png')

In [ ]:
pwd = "/content/drive/MyDrive/HDDS/Data/English/Augmented/"
cnt=12
for k in range(1,cnt):
  for i in range(2):
    for j in range(2):
      
      image = cv2.imread(pwd+str(k)+".png")
      test=noisy(SPIn[j],varr[i],"gauss",image)
      cv2.imwrite(pwd+str(cnt)+'.png',test)
      print(cnt)
      cnt=cnt+1
      test2=noisy(SPIn[j],varr[i],"s&p",image)
      cv2.imwrite(pwd+str(cnt)+'.png',test2)
      print(cnt)
      cnt=cnt+1
      test3=noisy(SPIn[j],varr[i],"poisson",image)
      cv2.imwrite(pwd+str(cnt)+'.png',test3)
      print(cnt)
      cnt=cnt+1
      test4=noisy(SPIn[j],varr[i],"speckle",image)
      cv2.imwrite(pwd+str(cnt)+'.png',test4)
      print(cnt)
      cnt=cnt+1

print(cnt)

In [ ]:
# we need to keep in mind aspect ratio so the image does
# not look skewed or distorted -- therefore, we calculate
# the ratio of the new image to the old image
l=(2550,3507)
total=0
pathh="/content/drive/MyDrive/HDDS/Data/English/Augmented/"
Savingpath="/content/drive/MyDrive/Cropped/"
for k in range(1,188):
  image = cv2.imread(pathh+str(k)+".png")
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  #r = 100.0 / image.shape[1]
  #dim = (100, int(image.shape[0] * r))
  
  # perform the actual resizing of the image and show it
  resized = cv2.resize(image, l, interpolation = cv2.INTER_AREA)
  #cv2.imshow("resized", resized)
  #plt.imshow(image)
  #plt.show()
  #print(image.shape)

  #Config value for this setting you should change it if needed.
  V1=90
  H1=70
  H2=260
  V2=260
  X=55

  for i in range(150) :
      cropped = resized[V1:V2,H1:H2]
      H1=H2+X
      H2=H2+X+170
      if ((i+1) % 10==0 and i != 0):
          H1=90
          H2=260
          V1=V2+X
          V2=V2+X+170  
      if(k==2 or k==1 or (k>=12 and k<=43)):
        #print("if 1 ///// k=",k)
        if((i+1)%10==0 and i!=0):
          cv2.imwrite(Savingpath+str(total-9)+'.png', cropped)
          total=total+1
        else:
         cv2.imwrite(Savingpath+str(total+1)+'.png', cropped) 
         total=total+1
      else: 

        cv2.imwrite(Savingpath+str(total)+'.png', cropped)
        total=total+1
      print(total) 

# ----Mannual----
#H H H H H
# 
# V
# V
# V
# V




In [ ]:
%cd /content/drive/MyDrive/Compressed

/content/drive/MyDrive/Compressed


In [ ]:
!sed -i 's/\r//' Compress.sh

In [ ]:
!bash Compress.sh

In [ ]:
image = cv2.imread("/content/drive/MyDrive/Compressed/28009.png",cv2.IMREAD_GRAYSCALE )

In [ ]:
image.shape

(28, 28)

In [ ]:
%cd "/content/drive/MyDrive"

/content/drive/MyDrive


In [ ]:
%pwd

'/content/drive/My Drive'

In [ ]:
#Script for making 1 channel image
from scipy import ndimage
counter =0
for i in range(28050) :
    ii = cv2.imread(str(i)+".png")
    gray_image = cv2.cvtColor(ii, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(str(i)+".png",gray_image)
    #print(gray_image.shape)
    #plt.imshow(gray_image)
    #plt.show()
